### Part 1
#### A description of the problem and a discussion of the background. (15 marks)


Clearly define a problem or an idea of your choice, where you would need to leverage the Foursquare location data to solve or execute. Remember that data science problems always target an audience and are meant to help a group of stakeholders solve a problem, so make sure that you explicitly describe your audience and why they would care about your problem.

This submission will eventually become your Introduction/Business Problem section in your final report. So I recommend that you push the report (having your Introduction/Business Problem section only for now) to your Github repository and submit a link to it.

### Part 2
####  A description of the data and how it will be used to solve the problem. (15 marks)


Describe the data that you will be using to solve the problem or execute your idea. Remember that you will need to use the Foursquare location data to solve the problem or execute your idea. You can absolutely use other datasets in combination with the Foursquare location data. So make sure that you provide adequate explanation and discussion, with examples, of the data that you will be using, even if it is only Foursquare location data.

This submission will eventually become your Data section in your final report. So I recommend that you push the report (having your Data section) to your Github repository and submit a link to it.

# Part 1 (Problem):
Where in Rio de Janeiro would it be best to open a restaurant. What is the place that has the most reviews, what location seems to be the most popular. Is there a type of restaurant that is more popular than another? 

The problem will be solved using the Foursquare API, looking at the usercount and tipcount. This problem will help everyone that wants to open up a restaurant in Rio de Janeiro and wants to know if it is a viable option where they intend on opening.

# Part 2 (Data):
The data that will be used is Foursquare data for Rio de Janeiro city. It will focus on venues of restaurant type. The popularity will be an indicator for how good an area is for a restaurant. 

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [35]:
# Get longitude and latitude for Toronto
address = 'Rio de Janeiro, Rio de Janeiro'

geolocator = Nominatim(user_agent="rio_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Rio de Janeiro is {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Rio de Janeiro is -22.9110137, -43.2093727.


In [25]:
# Set up Foursquare
CLIENT_ID = 'XBVDVFI3BB3LBMSPWBVCYUDG0AGUZRQP1MFUXOS0SKACIAGK' # your Foursquare ID
CLIENT_SECRET = 'HD34GAYDCO0GOUM1Z3NFCY4JH4FO0V5F4FMOLO4CEDSZO5AV' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: XBVDVFI3BB3LBMSPWBVCYUDG0AGUZRQP1MFUXOS0SKACIAGK
CLIENT_SECRET:HD34GAYDCO0GOUM1Z3NFCY4JH4FO0V5F4FMOLO4CEDSZO5AV


In [26]:
search_query = 'Restaurant'
radius = 50000
print(search_query + ' .... OK!')

Restaurant .... OK!


In [27]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=XBVDVFI3BB3LBMSPWBVCYUDG0AGUZRQP1MFUXOS0SKACIAGK&client_secret=HD34GAYDCO0GOUM1Z3NFCY4JH4FO0V5F4FMOLO4CEDSZO5AV&ll=-22.9110137,-43.2093727&v=20180605&query=Restaurant&radius=50000&limit=100'

In [28]:
results = requests.get(url).json()

In [29]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '52e81612bcbc57f1066b79f1', 'name': 'B...",False,56e04e1d38fa3ae11155b863,"Rua Riachuelo, 242",BR,Rio de Janeiro,Brasil,NaN,2146,"[Rua Riachuelo, 242, Rio de Janeiro, RJ, Brasil]","[{'label': 'display', 'lat': -22.914733, 'lng'...",-22.914733,-43.188829,NaN,NaN,RJ,Riá Restaurant,v-1559843522,NaN
1,"[{'id': '4bf58dd8d48988d16b941735', 'name': 'B...",False,4ece6782cc219860f521dd5d,NaN,BR,Rio de Janeiro,Brasil,NaN,1828,"[Rio de Janeiro, RJ, Brasil]","[{'label': 'display', 'lat': -22.9097338088607...",-22.909734,-43.191598,NaN,NaN,RJ,Restaurante Pitada,v-1559843522,NaN
2,"[{'id': '4bf58dd8d48988d1ce941735', 'name': 'S...",False,4bd22a3341b9ef3b3582fde5,"Av. Atlântica, 2936",BR,Rio de Janeiro,Brasil,NaN,7462,"[Av. Atlântica, 2936, Rio de Janeiro, RJ, Brasil]","[{'label': 'display', 'lat': -22.9749376592117...",-22.974938,-43.187433,NaN,NaN,RJ,Restaurante Mondego,v-1559843522,NaN
3,"[{'id': '4bf58dd8d48988d16b941735', 'name': 'B...",False,4d46d2732e326ea8e41afba6,"R. do Matoso, 30",BR,Rio de Janeiro,Brasil,Pç. da Bandeira,435,"[R. do Matoso, 30 (Pç. da Bandeira), Rio de Ja...","[{'label': 'display', 'lat': -22.9120948222039...",-22.912095,-43.213454,NaN,NaN,RJ,Restaurante Grão & Grão,v-1559843522,NaN
4,"[{'id': '4d4ae6fc7a7b7dea34424761', 'name': 'F...",False,56af89ea498e8dd1ce939e4a,"Rua Joaquim Palhares, 115",BR,Rio de Janeiro,Brasil,NaN,359,"[Rua Joaquim Palhares, 115, Rio de Janeiro, RJ...","[{'label': 'display', 'lat': -22.9139819676273...",-22.913982,-43.207982,NaN,20260-080,RJ,Restaurante Galeto Divino,v-1559843522,NaN


In [30]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Riá Restaurant,Bistro,"Rua Riachuelo, 242",BR,Rio de Janeiro,Brasil,NaN,2146,"[Rua Riachuelo, 242, Rio de Janeiro, RJ, Brasil]","[{'label': 'display', 'lat': -22.914733, 'lng'...",-22.914733,-43.188829,NaN,NaN,RJ,56e04e1d38fa3ae11155b863
1,Restaurante Pitada,Brazilian Restaurant,NaN,BR,Rio de Janeiro,Brasil,NaN,1828,"[Rio de Janeiro, RJ, Brasil]","[{'label': 'display', 'lat': -22.9097338088607...",-22.909734,-43.191598,NaN,NaN,RJ,4ece6782cc219860f521dd5d
2,Restaurante Mondego,Seafood Restaurant,"Av. Atlântica, 2936",BR,Rio de Janeiro,Brasil,NaN,7462,"[Av. Atlântica, 2936, Rio de Janeiro, RJ, Brasil]","[{'label': 'display', 'lat': -22.9749376592117...",-22.974938,-43.187433,NaN,NaN,RJ,4bd22a3341b9ef3b3582fde5
3,Restaurante Grão & Grão,Brazilian Restaurant,"R. do Matoso, 30",BR,Rio de Janeiro,Brasil,Pç. da Bandeira,435,"[R. do Matoso, 30 (Pç. da Bandeira), Rio de Ja...","[{'label': 'display', 'lat': -22.9120948222039...",-22.912095,-43.213454,NaN,NaN,RJ,4d46d2732e326ea8e41afba6
4,Restaurante Galeto Divino,Fried Chicken Joint,"Rua Joaquim Palhares, 115",BR,Rio de Janeiro,Brasil,NaN,359,"[Rua Joaquim Palhares, 115, Rio de Janeiro, RJ...","[{'label': 'display', 'lat': -22.9139819676273...",-22.913982,-43.207982,NaN,20260-080,RJ,56af89ea498e8dd1ce939e4a


In [31]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map